# Problem 2 Multiclass Logistic Regression

In [874]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [875]:
df=pd.read_csv("mnist_train.csv")
df=df.sort_values(['label'],ascending=True)

In [887]:
x_train=(df[df['label']<5]).drop('label',axis=1)
x_train=(x_train/255)

In [888]:
y=df[df['label']<5][['label']]

In [982]:
#One hot coding for traing lables
y_label=np.zeros((len(y),len(y.label.unique())))
y_=np.array(y)
for i in range(len(y.label.unique())):
    for j in range(len(y)):
        if y_[j]==y.label.unique()[i]:
            y_label[j,i]=1
        else:
            y_label[j,i]=0

# Multiclass Logistic Regression

In [978]:
class LogisticRegression():
    
    def __init__(self,learning_rate):
        self.learning_rate=learning_rate
    
    
    def softmax(self,z):
        return (np.exp(z)/np.sum(np.exp(z)))
    
    
    def fit(self,x,y):
        n_samples,n_features=x.shape
        self.bias=np.zeros((1,5))
        self.weights=np.zeros((n_features,5))
        
        for i in range(100):
            self.model=self.bias+np.dot(x,self.weights)
            y_hat=self.softmax(self.model)
            #print(y_hat.shape)
            
            #gradient descent
            db=np.array((1/len(x))*np.sum(y_hat-y))
            dw=(1/len(x))*(np.dot(x.T,(y_hat-y)))
            
            #update weights
            self.weights-=self.learning_rate*dw
            self.bias-=self.learning_rate*db
            
            #stopping criteria
            #if self.bias[0]<0.0001 and self.weights[0]<0.0001 and self.weights[1]<0.0001:
                #print('Number of iternations required to converge: ',i)
                #break
        
    def train(self,x):
        self.model=self.bias+np.dot(x,self.weights)
    
    def predict(self,x):
        self.y_probability=self.softmax(self.model)
        #y_predicted=[1 if y_probability>=0.5 else 0 for y_probability in self.y_probability]
        #self.y_predicted=np.asarray(y_predicted)
        self.y_predicted=np.array(self.y_probability.argmax(axis=1).reshape((len(x),1)))
        #return self.y_predicted
    
    def accuracy(self,x,y):
        count=0
        for i in range(len(x)):
            if self.y_predicted[i]==np.array(y)[i]:
                count+=1
        return (count/len(x))*100
    
    def roc_curve(self,x,y):
        y=np.array(y)
        tp=0
        tn=0
        fp=0
        fn=0
        for i in range(len(x)):
            if self.y_predicted[i]==1 and y[i]==1:
                tp+=1
            else:
             if self.y_predicted[i]==0 and y[i]==0:
                 tn+=1   
            if self.y_predicted[i]==1 and y[i]==0:
                fp+=1
            else:
             if self.y_predicted[i]==0 and y[i]==1:
                fn+=1
        true_positive_rate=tp/(tp+fn)
        false_positive_rate=fp/(fp+tn)
        print(tp,fn,fp,tn)
        return true_positive_rate,false_positive_rate
    
    def loss(self,x,y):
        loss= -(1/len(x)*np.sum(y*np.log(self.y_probability)+(1-y)*np.log(1-self.y_probability)))
        print('Loss: ',loss)
        
    def probability(self):
        return self.y_probability
    
    
    def predict_class(self):
        return self.y_predicted
    
    def roc(self,x,y):
        y=np.array(y)
        for j in range(5):
             for i in range(len(x)):
                self.model=self.bias+np.dot(x.iloc[i,:],self.weights)
                prob=self.softmax(self.model)
                if np.logical_and(prob[:,0]>=0.5,y[j]==1):
                    count+=1
                if np.logical_and(prob[:,0]<=0.5,y[j]==0):
                    count+=1

### Fitting model

In [1007]:
l=LogisticRegression(0.001)
l.fit(x_train,y_label)
l.train(x_train)
l.predict(x_train)
l.accuracy(x_train,y)

84.32474833311544

In [993]:
df_test=pd.read_csv("mnist_test.csv")
df_test=df_test.sort_values(['label'],ascending=True)

In [994]:
x_test=(df_test[df_test['label']<5]).drop('label',axis=1)
x_test=(x_test/255)

In [995]:
y_test=df_test[df_test['label']<5][['label']]

In [996]:
#One hot coding for testing 
y_label_test=np.zeros((len(y_test),len(y_test.label.unique())))
y_=np.array(y_test)
for i in range(len(y_test.label.unique())):
    for j in range(len(y_test)):
        if y_[j]==y_test.label.unique()[i]:
            y_label_test[j,i]=1
        else:
            y_label_test[j,i]=0

### Prediction on Testing Data Set

In [997]:
l.train(x_test)
l.predict(x_test)
l.accuracy(x_test,y_test)

85.46409807355516

In [998]:
l.roc_curve(x_test,y_test)

981 0 0 972


(1.0, 0.0)

### Precision, Recall and F1-Score

In [1006]:
from sklearn.metrics import classification_report
y_hat=l.predict_class()
print(classification_report(y_test,y_hat))

              precision    recall  f1-score   support

           0       0.75      0.99      0.86       980
           1       0.99      0.86      0.92      1135
           2       0.83      0.76      0.79      1032
           3       0.79      0.90      0.84      1010
           4       0.98      0.76      0.86       982

    accuracy                           0.85      5139
   macro avg       0.87      0.86      0.85      5139
weighted avg       0.87      0.85      0.86      5139

